In [1]:
import math
import argparse
import pandas as pd
import random
import csv
import time

import matplotlib.pyplot as plt
import torch
import numpy as np
from scipy.stats import boxcox

from mhcflurry import Class1AffinityPredictor

from torchvision import transforms, utils
from torch import nn 
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

import metrics
import data_transformation
import train_validation
import model_architecture
import rank_model
import utils
import unittest
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler

In [2]:
def test_modules(module_test):
    """Run unittest in imported files"""
    
    suite = unittest.TestLoader().loadTestsFromModule(module_test)
    unittest.TextTestRunner(verbosity=1).run(suite)

In [3]:
testmodules = [data_transformation.data_class_test(), metrics.metrics_test(), utils.utils_test()]
for test in testmodules:
    test_modules(test)

..
----------------------------------------------------------------------
Ran 2 tests in 17.458s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.014s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.089s

OK


In [2]:
params = {
    'batch_size': 64,
    'save_model_path': "ResNet_small.chkpt",
    'epoch': 200,
    'model': "ResNet_wide",
    'dropout': 0.2,
    "optimizer": "sgd", #adam sgd
    "learning_rate": 0.001,
    "adam_vanilla": True,
    'momentum': 0.9,
    'preproc': "log", #boxcox log
    'weight_decay': 0
}

In [5]:
#data_class = data_transformation.data_transformation(path_data = "/data/curated_training_data_no_mass_spec.csv",
#                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
#                                                     allele_name = None,
#                                                     quant_data = True,
#                                                     encoding = "one-hot")
#
#pep, mhc, target = data_class.__getitem__()
#
#pep = np.expand_dims(pep, axis=1)
#mhc = np.expand_dims(mhc, axis=1)
#inp = np.hstack((pep, mhc))
#print(inp.shape)
#
#X_train, X_test, y_train, y_test = train_test_split(inp, target.T, test_size=0.2, random_state=42)
data_class = data_transformation.data_transformation(path_data = "/data/data_curated_20180219/curated_training_data_no_mass_spec_dbscan.csv",
                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
                                                     allele_name = None,
                                                     quant_data = True,
                                                     encoding = "one-hot",
                                                     transformation = params['preproc'])


pep, mhc, target, dbscan = data_class.__getitem__()

if params['preproc'] == 'boxcox':
    target = boxcox(target.flatten(), 0.055).reshape(-1, 1)
else:
    target = target.T
pep = np.expand_dims(pep, axis=1)
mhc = np.expand_dims(mhc, axis=1)
inp = np.hstack((pep, mhc))
print(inp.shape)

np.random.seed(17) 
test_index = np.random.choice(np.where(dbscan.T == -1)[0],
                              size = int(np.round(dbscan.T.shape[0] * 0.2)),
                              replace = False)

X_train = inp[~np.isin(np.arange(len(inp)), test_index)]
X_test = inp[np.isin(np.arange(len(inp)), test_index)]
y_train = target[~np.isin(np.arange(len(target)), test_index)]
y_test = target[np.isin(np.arange(len(target)), test_index)]

if params['preproc'] == 'boxcox':
    scaler = StandardScaler()
    scaler.fit(y_train)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

if params['model'] == 'conv3x3':
    model = model_architecture.conv3x3(inputchannel = np.size(X_train, 3),
                                    L = np.size(X_train, 2),
                                    dropout = params['dropout'],
                                    dropoutearly = 0)#0.2)
elif params['model'] == 'convnet':
    model = model_architecture.convnet(inputchannel = np.size(X_train, 3),
                                L = np.size(X_train, 2),
                                dropout = params['dropout'])
elif params['model'] == 'ResNet18':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.BasicBlock, 
                                    num_blocks = [2,2,2,2],
                                    num_classes = 1,
                                    dropout = params['dropout'])
elif params['model'] == 'ResNet50':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.Bottleneck, 
                                    num_blocks = [3,4,6,3],
                                    num_classes = 1,
                                    dropout = params['dropout'])
elif params['model'] == 'ResNet_small':
    model = model_architecture.ResNet(inputchannel = np.size(X_train, 3),
                                    block = model_architecture.BasicBlock, 
                                    num_blocks = [1,1,1,1],
                                    num_classes = 1,
                                    dropout = params['dropout'])
  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Model parameters: " + str(pytorch_total_params) )
if torch.cuda.device_count() > 0:
    print('Using GPU' + str(utils.pick_gpu_lowest_memory()))
    device = torch.device('cuda:' + str(utils.pick_gpu_lowest_memory()))
else:
    print('Using CPU')
    device = torch.device('cpu')
    
model = model.to(device)
criterion = metrics.select_criterion('MSE')
if params['optimizer'] == "adam":
    if params['adam_vanilla']:
        optimizer = optim.Adam(
                      filter(lambda x: x.requires_grad, model.parameters()),
                      lr = params['learning_rate'],
                      weight_decay = params['weight_decay'])
    else:
        optimizer = optim.Adam(
                      filter(lambda x: x.requires_grad, model.parameters()),
                      lr = params['learning_rate'],
                      betas=(0.9, 0.98),
                      eps=1e-09,
                      weight_decay = params['weight_decay'])    
elif params['optimizer'] == "sgd":
    optimizer = optim.SGD(
                filter(lambda x: x.requires_grad, model.parameters()),
                lr=params['learning_rate'], 
                momentum=params['momentum'],
                weight_decay = params['weight_decay'])


train_data = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
test_data = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))

train_dataloader = torch.utils.data.DataLoader(train_data,
                              batch_size = params['batch_size'],
                              shuffle = True, 
                              drop_last = True)
eval_dataloader = torch.utils.data.DataLoader(test_data,
                              batch_size = params['batch_size'],
                              shuffle = True,
                              drop_last = True)

(156996, 2, 34, 20)


NameError: name 'model' is not defined

In [ ]:
valid_losss, train_losses, valid_accus, train_accus, true_valid, pred_valid = train_validation.start_training(params['save_model_path'], params['epoch'], model,
                                                                                                              train_dataloader, eval_dataloader, optimizer,
                                                                                                              device, criterion)

In [ ]:
valid_losss, train_losses, valid_accus, train_accus, true_valid, pred_valid = train_validation.continue_training(params['save_model_path'], params['epoch'], model, 
                                                                                                                 train_dataloader, eval_dataloader, optimizer, 
                                                                                                                 device, criterion)

In [ ]:
csv_columns = ['id', 'Loss', 'Accuracy', 'Model', 'Dropout',
               'Optimizer', 'Adam_vanilla', 'momentum', 
               'Learning_rate', 'Preprocess', 'Batch_size',
               'Weight_decay']

dict_data = []
time_exp = round(time.time())
for i in range(int(params['epoch'] / 5)):
    dict_data.append({'id': str(time_exp) + '_' + str(i*5+1),
                      'Loss': (valid_losss[i*5] / (np.max(y_test) - np.min(y_test))),
                      'Accuracy': valid_accus[i*5], 'Model': params['model'], 'Dropout': params['dropout'],
                      'Optimizer': params['optimizer'], 'Adam_vanilla': params['adam_vanilla'],
                      'momentum': params['momentum'], 'Learning_rate': params['learning_rate'],
                      'Preprocess': params['preproc'], 'Batch_size': params['batch_size'],
                      'Weight_decay': params['weight_decay']})

csv_file = "experiment_log.csv"
if os.path.isfile(csv_file):
    fileEmpty = os.stat(csv_file).st_size == 0
else:
    fileEmpty = True
try:
    with open(csv_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = csv_columns)
        if fileEmpty:
            writer.writeheader()  # file doesn't exist yet, write a header
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

## Predict new Values

In [3]:
def predict_mhc(allele_name, bootstrap_data):

    #data_class = data_transformation.data_transformation(path_data = "/data/curated_training_data_no_mass_spec.csv",
    #                                                     path_mhc = "/data/aligned_mhc_dataset.csv",
    #                                                     allele_name = None,
    #                                                     quant_data = True,
    #                                                     encoding = "one-hot")
    #
    #pep, mhc, target = data_class.__getitem__()
    #
    #pep = np.expand_dims(pep, axis=1)
    #mhc = np.expand_dims(mhc, axis=1)
    #inp = np.hstack((pep, mhc))
    #print(inp.shape)
    #
    #X_train, X_test, y_train, y_test = train_test_split(inp, target.T, test_size=0.2, random_state=42)
    data_class = data_transformation.data_transformation(path_data = bootstrap_data,#"/data/new_model_testset.csv",
                                                         path_mhc = "/data/aligned_mhc_dataset.csv",
                                                         allele_name = allele_name,
                                                         quant_data = True,
                                                         dbscan = False,
                                                         encoding = "one-hot",
                                                         transformation = params['preproc'])

    
    pep, mhc, target = data_class.__getitem__()

    if params['preproc'] == 'boxcox':
        target = boxcox(target.flatten(), 0.055).reshape(-1, 1)
    else:
        target = target.T
    pep = np.expand_dims(pep, axis=1)
    mhc = np.expand_dims(mhc, axis=1)
    inp = np.hstack((pep, mhc))
    print(inp.shape)

    if params['preproc'] == 'boxcox':
        scaler = StandardScaler()
        scaler.fit(y_train)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)

    if params['model'] == 'conv3x3':
        model = model_architecture.conv3x3(inputchannel = np.size(inp, 3),
                                        L = np.size(inp, 2),
                                        dropout = params['dropout'],
                                        dropoutearly = 0)#0.2)
    elif params['model'] == 'convnet':
        model = model_architecture.convnet(inputchannel = np.size(inp, 3),
                                    L = np.size(inp, 2),
                                    dropout = params['dropout'])
    elif params['model'] == 'ResNet18':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.BasicBlock, 
                                        num_blocks = [2,2,2,2],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'ResNet50':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.Bottleneck, 
                                        num_blocks = [3,4,6,3],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'ResNet_small':
        model = model_architecture.ResNet(inputchannel = np.size(inp, 3),
                                        block = model_architecture.BasicBlock, 
                                        num_blocks = [1,1,1,1],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'ResNet_wide':
        model = model_architecture.ResNet_wide(inputchannel = np.size(inp, 3),
                                        block = model_architecture.BasicBlock, 
                                        num_blocks = [1,1],
                                        num_classes = 1,
                                        dropout = params['dropout'])
    elif params['model'] == 'lstm':
        model = model_architecture.lstm(input_dim = 1360,
                                        batch_size = params['batch_size'])

    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print("Model parameters: " + str(pytorch_total_params) )
    if torch.cuda.device_count() > 0:
        print('Using GPU' + str(utils.pick_gpu_lowest_memory()))
        device = torch.device('cuda:' + str(utils.pick_gpu_lowest_memory()))
    else:
        print('Using CPU')
        device = torch.device('cpu')

    model = model.to(device)
    criterion = metrics.select_criterion('MSE')
    if params['optimizer'] == "adam":
        if params['adam_vanilla']:
            optimizer = optim.Adam(
                          filter(lambda x: x.requires_grad, model.parameters()),
                          lr = params['learning_rate'],
                          weight_decay = params['weight_decay'])
        else:
            optimizer = optim.Adam(
                          filter(lambda x: x.requires_grad, model.parameters()),
                          lr = params['learning_rate'],
                          betas=(0.9, 0.98),
                          eps=1e-09,
                          weight_decay = params['weight_decay'])    
    elif params['optimizer'] == "sgd":
        optimizer = optim.SGD(
                    filter(lambda x: x.requires_grad, model.parameters()),
                    lr=params['learning_rate'], 
                    momentum=params['momentum'],
                    weight_decay = params['weight_decay'])



    test_data = torch.utils.data.TensorDataset(torch.from_numpy(inp).float(), torch.from_numpy(target))

    test_dataloader = torch.utils.data.DataLoader(test_data,
                              batch_size = 1,
                              shuffle = False,
                              drop_last = False)

    a, b = train_validation.prediction_only(params['save_model_path'], model, test_dataloader, optimizer, device, criterion)
    out_pred = np.concatenate(a).ravel()
    out_true = np.concatenate(b).ravel()
    mymodel_out = pd.DataFrame(list(zip(out_pred, out_true)), columns =['pred', 'true']).apply(lambda y: rank_model.reverse_log_transformation(y))

    flurry_data = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis" + bootstrap_data)
    mask = (flurry_data['peptide'].str.len() >= 8) & (flurry_data['peptide'].str.len() <= 15)
    flurry_data = flurry_data.loc[mask]
    flurry_out = rank_model.mhcflurry_test(flurry_data, allele_name)
    
    error_dict_my, len_dict_my = rank_model.root_mean_squared(mymodel_out, allele_name, "my")
    error_dict_flurry, len_dict_flurry = rank_model.root_mean_squared(flurry_out, allele_name, "flurry")
    return(error_dict_my, len_dict_my, error_dict_flurry, len_dict_flurry)

## Model vergleich
Performance über bootstrap von allen testdaten nur mit mhc flurry --> außer ich bekomme mhcpan hier noch leicht rein <br>
Performance für einzelne Allele Vergleich zu MHCFlurry mit Bootstrap --> also nur Allele mit genug Daten ?vielleicht >100? <br>
Performance der allele nicht supported in MHCFlurry <br>
Performance verschiedene Längen Allele

In [82]:
# More MHCs are supported in my model
# No restriction for length of peptides -- mhcflurry 8 - 15 len
# Remove Mamu-A*01:01 because peptide bigger than 8-15 and not supported in mhcflurry

params['model'] = "ResNet18"
params['save_model_path'] = "final_model.chkpt"

# Compare only MHC supported in mhcflurry
allele_names = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/new_model_testset.csv").allele.unique()
predictor = Class1AffinityPredictor.load()

mhc_names = list(set(allele_names.tolist()) & set(predictor.supported_alleles))
combined_dict = {}
len_dict = {}

for i in mhc_names:
                    
    if i == "Mamu-A*01:01":
        print(i)
    else:
        error_my, len_my, error_flurry, len_flurry = predict_mhc(i, bootstrap_data = "/data/new_model_testset.csv")
        len_dict.update(len_my)
        len_dict.update(len_flurry)
        combined_dict.update(error_my)
        combined_dict.update(error_flurry)

(10, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.012474648157755534 min
(14, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.01708169380823771 min
(1, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.002498881022135417 min
(12, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.01825163761774699 min
(33, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


...

  - (Test) Elapse: 0.011667569478352865 min
(16, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.01641350587209066 min
(27, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.026496116320292154 min
(3, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.003667283058166504 min
(1, 2, 34, 20)
Model parameters: 11190657
Using CPU
Checkpoint found and loaded - Predict values
[ Epoch 1 ]


  - (Test) Elapse: 0.0015848477681477864 min


In [ ]:
# Bootstrapping 
from sklearn.utils import resample

import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout





params['model'] = "ResNet18"
params['save_model_path'] = "final_model.chkpt"

# Compare only MHC supported in mhcflurry
allele_names = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/new_model_testset.csv").allele.unique()
predictor = Class1AffinityPredictor.load()

mhc_names = list(set(allele_names.tolist()) & set(predictor.supported_alleles))
combined_dict = {}
len_dict = {}

df = pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/new_model_testset.csv")
df = df[df.allele.isin(mhc_names)]
values = df.values

# configure bootstrap
n_iterations = 1000
n_size = int(len(df) * 0.50)

# run bootstrap
stat_my = list()
stat_flurry = list()

for i in range(n_iterations):
    #if i % 1 == 0:
    #    print(i)
    print(i)
    # prepare train and test sets
    test = resample(values, n_samples=n_size)
    dataframe = pd.DataFrame.from_records(test,
                                         columns = ['Unnamed: 0', "allele", "peptide", "measurement_value", "original_allele", "measurement_type", "measurement_source"])
    path = "/data/test_bootstrap" + str(n_iterations) + ".csv"
    dataframe.to_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/test_bootstrap" + str(n_iterations) + ".csv")
    #print(path)
    combined_dict = {}
    len_dict = {}
    mhc_names = list(set(dataframe.allele.unique().tolist()) & set(predictor.supported_alleles))
    
    for i in mhc_names:

        if i == "Mamu-A*01:01":
            print(i)
        else:
            with HiddenPrints():
                error_my, len_my, error_flurry, len_flurry = predict_mhc(i, bootstrap_data = path)
            len_dict.update(len_my)
            len_dict.update(len_flurry)
            combined_dict.update(error_my)
            combined_dict.update(error_flurry)
        
    weighted_a = np.average(a = np.array([value for key, value in combined_dict.items() if 'my' in key.lower()]),
                    weights = np.array([value for key, value in len_dict.items() if 'my' in key.lower()]))
    weighted_b = np.average(a = np.array([value for key, value in combined_dict.items() if 'flurry' in key.lower()]),
                    weights = np.array([value for key, value in len_dict.items() if 'flurry' in key.lower()]))
        
    stat_my.append(weighted_a)
    stat_flurry.append(weighted_b)
    np.save("my_bootstrap.npy", np.array(stat_my))
    np.save("flurry_bootstrap.npy", np.array(stat_flurry))

0


Using TensorFlow backend.                                                                                              
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



1


2


3


...

729


Mamu-A*01:01


730


731


Mamu-A*01:01


In [ ]:
# plot scores
plt.hist(stat_my)
plt.show()
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(stat_my, p)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(stat_my, p)
print('%.1f confidence interval %.1f and %.1f' % (alpha*100, lower, upper))

In [ ]:
# plot scores
plt.hist(stat_flurry)
plt.show()
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(stat_flurry, p)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(stat_flurry, p)
print('%.1f confidence interval %.1f and %.1f' % (alpha*100, lower, upper))

In [25]:
#mean normal berechnen und einmal dann bootstrapping machen fürs finale modell

## Die extra MHC erwähnen? 

In [65]:
len(set(predictor.supported_alleles))

112

In [63]:
len(set(pd.read_csv("C:/Users/paul_/OneDrive/Desktop/master-thesis/data/aligned_mhc_dataset.csv").allele.unique().tolist()))

191

## Model performance

In [83]:
combined_dict

{'HLA-B*51:01my': 116594.51413010793,
 'HLA-B*51:01flurry': 116752.03927780298,
 'HLA-B*35:01my': 28816.1637933944,
 'HLA-B*35:01flurry': 25387.189435032473,
 'HLA-C*06:02my': 637.3681507443162,
 'HLA-C*06:02flurry': 6389.651623159466,
 'HLA-B*53:01my': 3691.0567744064297,
 'HLA-B*53:01flurry': 3996.717373484726,
 'HLA-A*01:01my': 348085.2374695057,
 'HLA-A*01:01flurry': 348418.4135603324,
 'HLA-B*15:01my': 37172.62194606468,
 'HLA-B*15:01flurry': 24822.65875695547,
 'HLA-B*18:01my': 16891.681180521693,
 'HLA-B*18:01flurry': 19141.08482740058,
 'HLA-A*02:01my': 24670.893884011683,
 'HLA-A*02:01flurry': 24509.24966538275,
 'HLA-A*68:02my': 6.2124156746669925,
 'HLA-A*68:02flurry': 24.83060400395638,
 'H-2-Dbmy': 10204.544930595703,
 'H-2-Dbflurry': 10417.937196912542,
 'HLA-B*27:02my': 33483.50813524559,
 'HLA-B*27:02flurry': 31696.841963361876,
 'HLA-A*11:01my': 7613.270171183466,
 'HLA-A*11:01flurry': 7002.518010106297,
 'HLA-B*38:01my': 4667.894930574795,
 'HLA-B*38:01flurry': 4770.0

In [84]:
len_dict

{'HLA-B*51:01my': 10,
 'HLA-B*51:01flurry': 10,
 'HLA-B*35:01my': 14,
 'HLA-B*35:01flurry': 14,
 'HLA-C*06:02my': 1,
 'HLA-C*06:02flurry': 1,
 'HLA-B*53:01my': 12,
 'HLA-B*53:01flurry': 12,
 'HLA-A*01:01my': 33,
 'HLA-A*01:01flurry': 33,
 'HLA-B*15:01my': 10,
 'HLA-B*15:01flurry': 10,
 'HLA-B*18:01my': 4,
 'HLA-B*18:01flurry': 4,
 'HLA-A*02:01my': 155,
 'HLA-A*02:01flurry': 155,
 'HLA-A*68:02my': 4,
 'HLA-A*68:02flurry': 4,
 'H-2-Dbmy': 135,
 'H-2-Dbflurry': 133,
 'HLA-B*27:02my': 11,
 'HLA-B*27:02flurry': 11,
 'HLA-A*11:01my': 19,
 'HLA-A*11:01flurry': 19,
 'HLA-B*38:01my': 12,
 'HLA-B*38:01flurry': 12,
 'HLA-B*08:01my': 4,
 'HLA-B*08:01flurry': 4,
 'HLA-B*27:05my': 8,
 'HLA-B*27:05flurry': 8,
 'HLA-A*68:01my': 4,
 'HLA-A*68:01flurry': 4,
 'HLA-B*44:03my': 14,
 'HLA-B*44:03flurry': 14,
 'HLA-B*07:02my': 47,
 'HLA-B*07:02flurry': 47,
 'HLA-A*03:01my': 30,
 'HLA-A*03:01flurry': 30,
 'HLA-B*44:02my': 29,
 'HLA-B*44:02flurry': 29,
 'H-2-Kbmy': 108,
 'H-2-Kbflurry': 106,
 'HLA-B*57:01my': 

In [85]:
weighted_a = np.average(a = np.array([value for key, value in combined_dict.items() if 'my' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'my' in key.lower()]))
a = np.mean(a = np.array([value for key, value in combined_dict.items() if 'my' in key.lower()]))
print(weighted_a)
print(a)

71800.6699773148
60599.590840471006


In [86]:
weighted_b = np.average(a = np.array([value for key, value in combined_dict.items() if 'flurry' in key.lower()]),
                        weights = np.array([value for key, value in len_dict.items() if 'flurry' in key.lower()]))
b = np.mean([value for key, value in combined_dict.items() if 'flurry' in key.lower()])
print(weighted_b)
print(b)

72132.82262110584
60594.88648959399


In [87]:
## Final -- with more data and all possible mhc also trained - more testdata - 5% valid
print((a/b - 1) * 100)
print((weighted_a/weighted_b - 1) * 100)

0.007763610346600558
-0.4604736536316434
